In [1]:
import tempfile
import re
import PyPDF2
import pdfplumber
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import pipeline

c:\Users\navee\OneDrive\Documents\bank_risk\new_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\navee\OneDrive\Documents\bank_risk\new_venv\Lib\site-packages\torch\cuda\__init__.py:749: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ..\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [73]:
tmp_file_path = r"scr\test1.pdf"

In [88]:
import pdfplumber

with pdfplumber.open(tmp_file_path) as pdf:
    full_text = ""
    for page in pdf.pages:
        full_text += page.extract_text()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
chunks = splitter.split_text(full_text)

In [89]:
chunks

['What is Loan Agreement?\nA loan agreement is a legally binding contract between a lender and a borrower that outlines the terms\nand conditions of a loan. It serves as a written agreement that establishes the rights and responsibilities\nof both parties in relation to the loan transaction.',
 'What is Cash Credit Loan?\nA cash credit loan, Is a short term financing option offered to bank.\nWhat are the Documents Required for a Cash Credit Loan?\nDocuments Required Cash Credit Loan are Proof of business registration, financial documents and',
 'Income tax returns, bank statements, and details of collateral.\nWho can avail of a Cash Credit Loan?\nCash Credit Loan : Any eligible person can avail Cash Credit Loan like Small and medium enterprises ,\nsole proprietorships, partnerships, and corporations can avail Cash Credit Loan.',
 'Documents Required for Personal Loans?\nDocuments Required for Personal Loans are Proof of identity, Proof of address, Proof of income etc...\nList of docume

In [90]:
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")


# Function to retrieve and clean context
def retrieve_and_clean_context(question, retriever):
    docs = retriever.get_relevant_documents(question)
    if docs:
        context = " ".join([doc.page_content for doc in docs])
        return context
    return ""

# Function to get an answer using the QA pipeline
def get_answer(question, retriever):
    context = retrieve_and_clean_context(question, retriever)
    if not context:
        return "No context retrieved."
    result = qa_pipeline(question=question, context=context)
    return result['answer']

model_name = 'sentence-transformers/all-MiniLM-L6-v2'
embeddings = HuggingFaceEmbeddings(model_name=model_name)
vectorstore = FAISS.from_texts(chunks, embeddings)
retriever = vectorstore.as_retriever()

c:\Users\navee\OneDrive\Documents\bank_risk\new_venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [91]:
question = "What is Cash Credit Loan ?"
if question:
    answer = get_answer(question, retriever)
    print(f"Question: {question}")
    print(f"Answer: {answer}")

Question: What is Cash Credit Loan ?
Answer: a short term financing option offered to bank


In [92]:
questions = [
    "What is Loan Agreement?",
    "What is Cash Credit Loan?",
    "What are the Documents Required for a Cash Credit Loan?",
    'Who can avail of a Cash Credit Loan?',
    "Documents Required For Personal Loans?",
    "List of documents for Salaried personal loans?",
    "Which documents could be used as income proof?",
    "What are the documentation process for a pre-approved Personal Loan?",
    "How fast is the Personal Loan documentation process?",
    "Do we have to submit my original KYC documents?"
]

for question in questions:
    answer = get_answer(question, retriever)
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print("*************************\n")

Question: What is Loan Agreement?
Answer: a legally binding contract between a lender and a borrower
*************************

Question: What is Cash Credit Loan?
Answer: a short term financing option offered to bank
*************************

Question: What are the Documents Required for a Cash Credit Loan?
Answer: Proof of identity, Proof of address, Proof of income
*************************

Question: Who can avail of a Cash Credit Loan?
Answer: Any eligible person
*************************

Question: Documents Required For Personal Loans?
Answer: Proof of identity, Proof of address, Proof of income
*************************

Question: List of documents for Salaried personal loans?
Answer: employee ID card, salary slips
*************************

Question: Which documents could be used as income proof?
Answer: payment proof, salary slip, bank statements
*************************

Question: What are the documentation process for a pre-approved Personal Loan?
Answer: no need of docum

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

chunks = splitter.split_documents(full_text)
print(f"Number of chunks: {len(chunks)}")
print(f"Length of a chunk: {len(chunks[1].page_content)}")
print("Content of a chunk:", chunks[1].page_content)

Number of chunks: 38
Length of a chunk: 456
Content of a chunk: MONOPOLY game, refer to the Classic Rules beginning on the next page. 
If you already know how to play and want to use the Speed Die, just 
read the section below for the additional Speed Die rules. 
SPEED DIE RULES 
Learnins how to Play with the S~eed Die IS as 
/ 
fast as playing with i't. 
1. When starting the game, hand out an extra $1,000 to each player 
(two $5005 should work). The game moves fast and you'll need 
the extra cash to buy and build.


In [10]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

model_name = 'all-MiniLM-L6-v2'
embeddings = HuggingFaceEmbeddings(model_name=model_name)
vectorstore = FAISS.from_documents(chunks, embeddings)

In [32]:
retriever = vectorstore.as_retriever()
retriever.invoke("go to jail?")

[Document(metadata={'source': 'scr\\monopoly.pdf', 'page': 4}, page_content='fine. You then get out of Jail and immediately move forward the number \nof spaces shown by your throw. \nEven though you are in Jail, you may buy and sell property, buy and \nsell houses and hotels and collect rents.'),
 Document(metadata={'source': 'scr\\monopoly.pdf', 'page': 4}, page_content='When you are sent to Jail you cannot \ncollect your $200 salary in that move \nsince, regardless of where your token \nis on the board, you must move it \ndirectly into Jail. Your turn ends when \nyou are sent to Jail. \nIf you are not "sent" to Jail but in the ordinary course of play land on \nthat space, you are "Just Visiting," you incur no penalty, and you move \nahead in the usual manner on your next turn. \nYou get out of Jail by.. .(I) throwing doubles on any of your next'),
 Document(metadata={'source': 'scr\\monopoly.pdf', 'page': 4}, page_content='your total worth to the Bank. Your total worth is all your ca

In [21]:
from transformers import pipeline

# Load the model and tokenizer
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# Define the context and question
context = "Anna's sister is Susan"
question = "Who is Susan's sister?"

# Get the answer from the model
result = qa_pipeline(question=question, context=context)

# Print the answer
print(result['answer'])

Anna


In [45]:
question = "how much money user needed to start the game?"

def retrieve_context(question):
    context = retriever.invoke(question)
    return context

context = retrieve_context(question)

context

c:\Users\navee\OneDrive\Documents\bank_risk\new_venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[Document(metadata={'source': 'scr\\monopoly.pdf', 'page': 2}, page_content="Each player is given $1,500 divided as follows: P each of $500s, \n$100~ and $50~; 6 $40~; 5 each of $105, $5~ and $Is. \nAll remaining money and other equipment go to the Bank. Stack the .. \nBank's money on edge in the compartments in the plastic Banker's tray. \nBANKER. Select as Banker a player who will also \nmake a good Auctioneer A Banker who plays \n~n the game must keep hislher personal funds \nseparate from those of the Bank. When more than"),
 Document(metadata={'source': 'scr\\monopoly.pdf', 'page': 0}, page_content="the extra cash to buy and build. \n2. Do not use the Speed Die until you've landed on or passed over \nGO for the first time. Once you collect that first $200 salary, you'll \nuse the Speed Die for the rest of the game. This means that some \nplayers will start using the die before others. \n3. Once you start using the Speed Die, roll it along with the two \nwhite dice on your turn. Th

In [116]:
question = "how much money user needed to start the game?"

def retrieve_context(question, retriever):
    docs = retriever.get_relevant_documents(question)
    if docs:
        # Combine the most relevant documents to use as context
        context = " ".join([doc.page_content for doc in docs])
        return context
    return ""

retriever = vectorstore.as_retriever()

context = retrieve_context(question, retriever)
context

c:\Users\navee\OneDrive\Documents\bank_risk\new_venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Each player is given $1,500 divided as follows: P each of $500s, \n$100~ and $50~; 6 $40~; 5 each of $105, $5~ and $Is. \nAll remaining money and other equipment go to the Bank. Stack the .. \nBank\'s money on edge in the compartments in the plastic Banker\'s tray. \nBANKER. Select as Banker a player who will also \nmake a good Auctioneer A Banker who plays \n~n the game must keep hislher personal funds \nseparate from those of the Bank. When more than the extra cash to buy and build. \n2. Do not use the Speed Die until you\'ve landed on or passed over \nGO for the first time. Once you collect that first $200 salary, you\'ll \nuse the Speed Die for the rest of the game. This means that some \nplayers will start using the die before others. \n3. Once you start using the Speed Die, roll it along with the two \nwhite dice on your turn. Then do the following depending on \nwhat you rolled. \n1, 2, or 3: Add this number to the roll of the two white required on mortgages. \nThe Bank collect

In [123]:
import re
from transformers import pipeline

# Initialize the question-answering pipeline with a pre-trained model
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# Function to clean the context
def clean_context(context):
    cleaned_context = context.replace("\n", " ") 
    cleaned_context = cleaned_context.replace("\\", "")  
    cleaned_context = re.sub(r"[.\;\:\~\"\'\()]", "", cleaned_context)
    cleaned_context = re.sub(r"\s+", " ", cleaned_context).strip()
    return cleaned_context

# Function to retrieve and clean context
def retrieve_and_clean_context(question, retriever):
    docs = retriever.get_relevant_documents(question)
    if docs:
        context = " ".join([doc.page_content for doc in docs])
        cleaned_context = clean_context(context)
        return cleaned_context
    return ""

# Function to get an answer using the QA pipeline
def get_answer(question, retriever):
    context = retrieve_and_clean_context(question, retriever)
    if not context:
        return "No context retrieved."
    result = qa_pipeline(question=question, context=context)
    return result['answer']

# Example usage
question = "How much money is needed to start the game?"
retriever = vectorstore.as_retriever()
answer = get_answer(question, retriever)
print(f"Question: {question}")
print(f"Answer: {answer}")


c:\Users\navee\OneDrive\Documents\bank_risk\new_venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Question: How much money is needed to start the game?
Answer: $1,000


In [107]:
from transformers import pipeline

# Initialize the question-answering pipeline with a pre-trained model
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# Define function to retrieve context
def retrieve_context(question):
    doc = retriever.invoke(question)
    return getattr(doc, 'page_content', str(doc))  # Retrieve 'page_content' or fallback to str(doc)

# Define function to get an answer
def get_answer(question):
    context = retrieve_context(question)
    if not context:
        return "No context retrieved."
    result = qa_pipeline(question=question, context=context)
    return result['answer']

# Example usage
question = "How much money is needed to start the game?"
answer = get_answer(question)
print(f"Question: {question}")
print(f"Answer: {answer}")

c:\Users\navee\OneDrive\Documents\bank_risk\new_venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Question: How much money is needed to start the game?
Answer: $1,000


In [109]:
from transformers import pipeline

# Initialize the question-answering pipeline with a BERT-large model
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

# Define function to retrieve context
def retrieve_context(question):
    doc = retriever.invoke(question)
    return getattr(doc, 'page_content', str(doc))

# Define function to get an answer
def get_answer(question):
    context = retrieve_context(question)
    if not context:
        return "No context retrieved."
    result = qa_pipeline(question=question, context=context)
    return result['answer']

# Example usage
question = "How much money is needed to start the game?"
answer = get_answer(question)
print(f"Question: {question}")
print(f"Answer: {answer}")


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\navee\OneDrive\Documents\bank_risk\new_venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default

Question: How much money is needed to start the game?
Answer: $1,000


In [113]:
from transformers import pipeline

# Initialize the question-answering pipeline with the RoBERTa model
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Define function to retrieve context
def retrieve_context(question):
    doc = retriever.invoke(question)
    return getattr(doc, 'page_content', str(doc))

# Define function to get an answer
def get_answer(question):
    context = retrieve_context(question)
    if not context:
        return "No context retrieved."
    result = qa_pipeline(question=question, context=context)
    return result['answer']

# Example usage
question = "How much money is needed to start the game?"
answer = get_answer(question)
print(f"Question: {question}")
print(f"Answer: {answer}")


c:\Users\navee\OneDrive\Documents\bank_risk\new_venv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\navee\.cache\huggingface\hub\models--deepset--roberta-base-squad2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\navee\OneDrive\Documents\bank_risk\new_venv\Lib\site-packages\transformers\

Question: How much money is needed to start the game?
Answer: $1,000


In [114]:
from transformers import pipeline

# Initialize the question-answering pipeline with the BERT-large model
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

# Define function to retrieve context
def retrieve_context(question):
    doc = retriever.invoke(question)
    return getattr(doc, 'page_content', str(doc))

# Define function to get an answer
def get_answer(question):
    context = retrieve_context(question)
    if not context:
        return "No context retrieved."
    result = qa_pipeline(question=question, context=context)
    return result['answer']

# Example usage
question = "How much money is needed to start the game?"
answer = get_answer(question)
print(f"Question: {question}")
print(f"Answer: {answer}")

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\navee\OneDrive\Documents\bank_risk\new_venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default

Question: How much money is needed to start the game?
Answer: $1,000


In [105]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model and tokenizer
model_name = "gpt2-medium"  # Using the "gpt2-medium" model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

def generate_response_with_context(context, question):
    # Define the prompt template
    template = """
    Use the following context to answer the question. Provide only the answer and nothing else.
    
    Do not ask questions back. If you can't find the answer in the context, say "I don't know."
    
    Never question the user.

    Context: {context}

    Question: {question}

    Answer:
    """
    
    # Format the prompt using the template
    prompt = template.format(context=context, question=question)
    
    # Encode the prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    
    # Generate text using the model
    outputs = model.generate(
        inputs,
        max_length=150,  # Limit the maximum length
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=2,  # Prevent repetition
        top_k=30,  # Top-k sampling
        top_p=0.85,  # Top-p (nucleus) sampling
        early_stopping=True  # Stop generation early if possible
    )
    
    # Decode the generated text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    
    # Post-process response to keep only the answer
    # Extract the answer following the "Answer:" keyword
    if "Answer:" in response:
        response = response.split("Answer:")[1].strip()
    else:
        # Handle the case where "Answer:" was not included in the response
        response = response.split("\n")[0].strip()
    
    # Ensure the response is relevant and concise
    if not response or response.lower() == "i don't know":
        response = "I don't know"
    
    return response

# Example usage
context = "Anna is Susan's sister, Raja is Sofi's brother, and Anna and Raja are playing in the playground."
question = "Who is Susan's sister?"
response = generate_response_with_context(context, question)
print(f"Context: {context}")
print(f"Question: {question}")
print(f"Response: {response}")


c:\Users\navee\OneDrive\Documents\bank_risk\new_venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\navee\OneDrive\Documents\bank_risk\new_venv\Lib\site-packages\transformers\generation\configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\navee\OneDrive\Documents\bank_risk\new_venv\Lib\site-packages\transformers\generation\configuration_utils.py:589: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `30` -- this flag is only used in sample-based generation

Context: Anna is Susan's sister, Raja is Sofi's brother, and Anna and Raja are playing in the playground.
Question: Who is Susan's sister?
Response: Susan is the sister of SoFi. 


Question: What is sofi?
